In [4]:
# !pip install datasets

In [5]:
# %%
# 1. Imports
import pandas as pd
import numpy as np
import torch
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
from sklearn.metrics import classification_report
import joblib
import hashlib
import os

In [6]:
# %%
# 2. Load IMDb dataset (no login, local use only)
dataset = load_dataset("imdb", cache_dir="./data")

df_train = dataset['train'].to_pandas()
df_test = dataset['test'].to_pandas()

df_train = df_train[['text', 'label']]
df_test = df_test[['text', 'label']]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [7]:
# %%
# 3. Tokenization
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)

train_encodings = dataset['train'].map(tokenize, batched=True)
test_encodings = dataset['test'].map(tokenize, batched=True)

train_encodings.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_encodings.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [8]:
# %%
# 4. Load DistilBERT model
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
# %%
# 5. Training setup
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="no",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    logging_dir='./logs',
    logging_steps=100
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_encodings,
    eval_dataset=test_encodings
)

trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: boutaina247 (boutaina247-ia-school) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
1,0.242800,0.231926


TrainOutput(global_step=3125, training_loss=0.3092559410095215, metrics={'train_runtime': 1444.2503, 'train_samples_per_second': 17.31, 'train_steps_per_second': 2.164, 'total_flos': 3311684966400000.0, 'train_loss': 0.3092559410095215, 'epoch': 1.0})

In [10]:
# %%
# 6. Evaluation
preds = trainer.predict(test_encodings)
y_pred = np.argmax(preds.predictions, axis=1)
y_true = preds.label_ids

print(classification_report(y_true, y_pred, target_names=["negative", "positive"]))

# %%
# 7. Save model and tokenizer
os.makedirs("models", exist_ok=True)

model_path = "models/distilbert_model"
tokenizer_path = "models/tokenizer"

model.save_pretrained(model_path)
tokenizer.save_pretrained(tokenizer_path)

joblib.dump(tokenizer, "models/tokenizer.joblib")

print("✅ Model and tokenizer saved.")

# %%
# 8. Hash model folder
def hash_dir(path):
    sha = hashlib.sha256()
    for root, dirs, files in os.walk(path):
        for f in sorted(files):
            with open(os.path.join(root, f), 'rb') as file:
                while chunk := file.read(8192):
                    sha.update(chunk)
    return sha.hexdigest()

model_hash = hash_dir(model_path)

with open("models/model_hash.txt", "w") as f:
    f.write(model_hash)

print(f"🧾 Model hash saved: {model_hash}")


              precision    recall  f1-score   support

    negative       0.92      0.93      0.92     12500
    positive       0.93      0.92      0.92     12500

    accuracy                           0.92     25000
   macro avg       0.92      0.92      0.92     25000
weighted avg       0.92      0.92      0.92     25000

✅ Model and tokenizer saved.
🧾 Model hash saved: 0c042fb3fb23bf11a0e3f6e71f7cb306feb05cb0a79e0a6c7b847dd6944a9f15


In [11]:
import joblib

# Save model and tokenizer
joblib.dump(model, "distilbert_model.joblib")
joblib.dump(tokenizer, "tokenizer_new.joblib")


['tokenizer_new.joblib']

In [12]:
import torch

# Save the model using torch.save()
torch.save(model.state_dict(), "distilbert_model.pth")
